In [1]:
import calliope
import pandas as pd
import plotly.express as px

# We increase logging verbosity
calliope.set_log_verbosity("INFO", include_solver_output=False)

In [2]:
model = calliope.Model('model.yaml')

[2024-12-12 03:15:10] INFO     Model: initialising
[2024-12-12 03:15:10] INFO     Model: preprocessing stage 1 (model_run)


ModelError: Errors during validation of the tech definition at node `Iceland` dictionary.:
 * techs.aluminum_supply: 'base_tech' is a required property
 * techs.aluminum_supply: {} is not valid under any of the given schemas

In [ ]:
model.inputs

In [ ]:
model.inputs.flow_cap_max.to_series().dropna()

In [ ]:
model.inputs.sink_use_equals.sum(
    "timesteps", min_count=1, skipna=True
).to_series().dropna()

In [ ]:
model.build(force=True)
model.solve()

In [ ]:
model.results

In [ ]:
costs = model.results.cost.to_series().dropna()
costs.head()

In [ ]:
lcoes = model.results.systemwide_levelised_cost.to_series().dropna()
lcoes

In [ ]:
# We set the color mapping to use in all our plots by extracting the colors defined in the technology definitions of our model.
colors = model.inputs.color.to_series().to_dict()

In [ ]:
df_electricity = (
    (model.results.flow_out.fillna(0) - model.results.flow_in.fillna(0))
    .sel(carriers="aluminum")
    .sum("nodes")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow in/out (kWh)")
    .reset_index()
)
df_electricity_demand = df_electricity[df_electricity.techs == "aluminum_demand"]
df_electricity_other = df_electricity[df_electricity.techs != "aluminum_demand"]

print(df_electricity.head())

fig1 = px.bar(
    df_electricity_other,
    x="timesteps",
    y="Flow in/out (kWh)",
    color="techs",
    color_discrete_map=colors,
)
fig1.add_scatter(
    x=df_electricity_demand.timesteps,
    y=-1 * df_electricity_demand["Flow in/out (kWh)"],
    marker_color="black",
    name="aluminum_demand",
)

In [ ]:
df_electricity = (
    (model.results.flow_out.fillna(0) - model.results.flow_in.fillna(0))
    .sel(carriers="aluminum")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow in/out (kWh)")
    .reset_index()
)
df_electricity_demand = df_electricity[df_electricity.techs == "aluminum_demand"]
df_electricity_other = df_electricity[df_electricity.techs != "aluminum_demand"]

print(df_electricity.head())

node_order = df_electricity.nodes.unique()

fig = px.bar(
    df_electricity_other,
    x="timesteps",
    y="Flow in/out (kWh)",
    facet_row="nodes",
    color="techs",
    category_orders={"nodes": node_order},
    height=1000,
    color_discrete_map=colors,
)

showlegend = True
# we reverse the node order (`[::-1]`) because the rows are numbered from bottom to top.
for idx, node in enumerate(node_order[::-1]):
    demand_ = df_electricity_demand.loc[
        df_electricity_demand.nodes == node, "Flow in/out (kWh)"
    ]
    if not demand_.empty:
        fig.add_scatter(
            x=df_electricity_demand.loc[
                df_electricity_demand.nodes == node, "timesteps"
            ],
            y=-1 * demand_,
            row=idx + 1,
            col="all",
            marker_color="black",
            name="Demand",
            legendgroup="demand",
            showlegend=showlegend,
        )
        showlegend = False
fig.update_yaxes(matches=None)
fig.show()

In [ ]:
df_capacity = (
    model.results.flow_cap.where(model.results.techs != "aluminum_demand")
    .sel(carriers="aluminum")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow capacity (kW)")
    .reset_index()
)

print(df_capacity.head())

fig = px.bar(
    df_capacity,
    x="nodes",
    y="Flow capacity (kW)",
    color="techs",
    color_discrete_map=colors,
)
fig.show()

In [ ]:
df_coords = model.inputs[["latitude", "longitude"]].to_dataframe().reset_index()
df_capacity = (
    model.results.flow_cap.where(model.inputs.base_tech == "transmission")
    .sel(carriers="aluminum")
    .to_series()
    .where(lambda x: x != 0)
    .dropna()
    .to_frame("Flow capacity (kW)")
    .reset_index()
)
df_capacity_coords = pd.merge(df_coords, df_capacity, left_on="nodes", right_on="nodes")

fig = px.line_mapbox(
    df_capacity_coords,
    lat="latitude",
    lon="longitude",
    color="techs",
    hover_data="Flow capacity (kW)",
    zoom=3,
    height=300,
)
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=4,
    mapbox_center_lat=df_coords.latitude.mean(),
    mapbox_center_lon=df_coords.longitude.mean(),
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
)